# Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial - Inteligencia Artificial para Juegos (Game AI) - Sesión 2 (Tarea)

<font color='orange'>Entorno del 8-puzzle</font>

El presente laboratorio aborda la creacion de un agente de busqueda para solucionar cualquier instancia del entorno del 8-puzzle. Para ello se debe definir completamente el problema de busqueda. Un Algoritmo general de búsqueda con memoria de nodos expandidos ya se encuentra implementado (graph_search). Al final del notebook deberas responder a las preguntas planteadas.
Al final del notebook deberas responder a las preguntas planteadas. 

## Definición de entornos

### Clase <b>Thing</b>

  Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>. (No editar)  

In [0]:
class Thing(object):
    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

### Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que guarda el desempeño del agente en su ambiente (desempeño visto desde el agente).

In [0]:
import collections
import random

class Agent(Thing):
    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

### Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [0]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepcion que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una accion 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localizacion especificada."""
        return None

    def is_done(self):
        """Retorna True si no hay ningun agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade una cosa thing al entorno en la localizacion location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

### Clase <b>EightpuzzleEnvironment </b>

En esta clase se implementa el entorno del 8-puzzle. Un agente en este entorno percibe el estado del entorno como  string de 9 caracteres: los tres primeros caracteres representan las posiciones de la fila de arriba,  los tres siguientes caracteres la fila del medio y los 3 ultimos caracteres las posiciones de abajo del puzzle. El caracter * representa la posición en blanco.  

In [0]:
class EightpuzzleEnvironment(Environment):

    def __init__(self, initial_state):
        super().__init__()
        self.status = initial_state
        
    def thing_classes(self):
        return [EightpuzzleReflexAgent]

    def percept(self, agent):
        """Retorna el estado del ambiente (las piezas que estan en cada posicion)"""
        return self.status

    def execute_action(self, agent, action):
        """Implementa el MAPA De TRANSICION: Cambia la posicion de las piezas de acuerdo a la accion solicitada del blanco; 
        Cada accion valida debe provocar una disminución de desempeño en 1 unidad """
        
        state = list(self.status)
        iblank = state.index('*')    # obtiene el indice del casillero en blanco (representado con *)
        
        if action == 'Right':
            if (iblank != 2 and iblank != 5 and iblank != 8):
                state[iblank], state[iblank+1] = state[iblank+1], state[iblank]
                agent.performance -= 1
                
        elif action == 'Left':
            if (iblank != 0 and iblank != 3 and iblank != 6):
                state[iblank-1], state[iblank] = state[iblank], state[iblank-1]
                agent.performance -= 1
                
        elif action == 'Up':
            if (iblank > 2):
                state[iblank-3], state[iblank] = state[iblank], state[iblank-3]
                agent.performance -= 1
                
        elif action == 'Down':
            if (iblank < 6):
                state[iblank], state[iblank+3] = state[iblank+3], state[iblank]
                agent.performance -= 1
                
        self.status = ''.join(state) 

## Algoritmos de Búsqueda

### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [0]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

    def value(self, state):
        """En problemas de optimizacion, cada estado tiene un valor. Algoritmos
        como Hill-climbing intentan maximizar este valor."""
        raise NotImplementedError

### Clase <b>Node</b>

Estructura de datos para almacenar la informacion de un nodo en un <b>arbol de busqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la accion que nos llevo al presente nodo y el costo total del camino desde el nodo raiz hasta este nodo.

In [0]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def __lt__(self, node):
        return self.state < node.state
    
    def __eq__(self, other): 
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state
    
    def __repr__(self):
        return "<Node {}>".format(self.state)
    
    def __hash__(self):
        return hash(self.state)

### <b> Define una cola tipo FIFO First-In-First-Out (para BFS)</b> 

In [0]:
from collections import deque

class FIFOQueue(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

### <b> Frontera tipo cola de prioridad ordenada por una funcion de costo (para best_first_graph_search)</b> 

In [0]:
import heapq
class FrontierPQ:
    "Una Frontera ordenada por una funcion de costo (Priority Queue)"
    
    def __init__(self, initial, costfn=lambda node: node.path_cost):
        "Inicializa la Frontera con un nodo inicial y una funcion de costo especificada (por defecto es el costo de camino)."
        self.heap   = []
        self.states = {}
        self.costfn = costfn
        self.add(initial)
    
    def add(self, node):
        "Agrega un nodo a la frontera."
        cost = self.costfn(node)
        heapq.heappush(self.heap, (cost, node))
        self.states[node.state] = node
        
    def pop(self):
        "Remueve y retorna el nodo con minimo costo."
        (cost, node) = heapq.heappop(self.heap)
        self.states.pop(node.state, None) # remove state
        return node
    
    def replace(self, node):
        "node reemplaza al nodo de la Fontera que tiene el mismo estado que node."
        if node.state not in self:
            raise ValueError('{} no tiene nada que reemplazar'.format(node.state))
        for (i, (cost, old_node)) in enumerate(self.heap):
            if old_node.state == node.state:
                self.heap[i] = (self.costfn(node), node)
                heapq._siftdown(self.heap, 0, i)
                return

    def __contains__(self, state): return state in self.states
    
    def __len__(self): return len(self.heap)

### <b>Algoritmo general de búsqueda con memoria de nodos expandidos (Graph Search)</b>

Algoritmo de general de busqueda ciega con memoria de estados visitados. El argumento frontier debe ser una cola vacia. Si  frontier es tipo FIFO hace busqueda en amplitud (BFS), si la frontier es una pila hará busqueda en profundidad (DFS). Devuelve el nodo solucion y una lista de nodos visitados durante la busqueda.

In [0]:
def graph_search(problem, frontier):
    frontier.append(Node(problem.initial))
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la busqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

### <b> Algoritmo de Busqueda por la mejor opción (Best-First-Graph-Search) </b> 
Algoritmo general de busqueda con información. La frontera es una cola de prioridad ordenada por la funcion de evaluacion f. Devuelve el nodo solucion y una lista de nodos visitados durante la busqueda 

In [0]:
def best_first_graph_search(problem, f):
    """Busca el objetivo expandiendo el nodo de la frontera con el menor valor de la funcion f. Memoriza estados visitados
    Antes de llamar a este algoritmo hay que especificar La funcion f(node). Si f es node.depth tenemos Busqueda en Amplitud; 
    si f es node.path_cost tenemos Busqueda  de Costo Uniforme. Si f es una heurística tenemos Busqueda Voraz;
    Si f es node.path_cost + heuristica(node) tenemos A* """

    frontier = FrontierPQ( Node(problem.initial), f )  # frontera tipo cola de prioridad ordenada por f
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la busqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)        
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        for action in problem.actions(node.state):
            child = node.child_node(problem, action)
            if child.state not in explored and child.state not in frontier:
                frontier.add(child)
            elif child.state in frontier:
                incumbent = frontier.states[child.state] 
                if f(child) < f(incumbent):
                    frontier.replace(child)

### <b> Algoritmo A* </b> 
A* es un caso especial de best_first_graph_search con f = path_cost + heuristic

In [0]:
def astar_search(problem, heuristic):
    f = lambda node: node.path_cost + heuristic(node, problem)
    return best_first_graph_search(problem, f)

def nullheuristic(node, problem):   # heurística nula (A* se convierte en busqueda de costo uniforme)
    return 0

### <b>Clase que implementa el programa del agente que busca y ejecuta soluciones en el entorno 8-puzzle</b>


In [0]:
class EightpuzzleSearchProgram:
    def __init__(self, goal_state, search_method):
        self.goal = goal_state
        self.method = search_method
        self.seq = []  # lista de acciones a ejecutar, inicialmente vacia
        
    def __call__(self, percept):
        state = percept
        if state == self.goal:    # Si el ambiente esta en el estado objetivo no hace nada
            return 'None'
        if not self.seq:  # si la lista de acciones esta vacia
            print('Agente buscara solucion al 8-puzzle: estado_inicial = {}. estado_objetivo={}'.format(state,self.goal))
            search_problem = EightpuzzleSearchProblem(state, self.goal)
            if self.method == 'bfs':
                goal_node, visited_nodes = graph_search(search_problem, FIFOQueue()) # frontera es una cola FIFO  
            elif self.method == 'dfs':
                goal_node, visited_nodes = graph_search(search_problem, []) # frontera es una pila ([] es una pila en Python)
            elif self.method == 'ucs':   # uniform cost search
                goal_node, visited_nodes = astar_search(search_problem, nullheuristic)
            else:
                raise NotImplementedError
                        
            if goal_node == None: # sin solucion
                print('No se encontro solucion para el 8-puzzle con metodo {}'.format(self.method) )
                return 'None'
            
            self.seq = goal_node.solution()
            print('Agente planeo una solucion al 8-puzzle con {}: Seq = {}. Nodos visitados={}. Costo Solucion = {}'.format(self.method, self.seq, len(visited_nodes),goal_node.path_cost)) 
        
        action = self.seq.pop(0)       
        return action 

## Clases para el entorno 8-puzzle

### <b> Clase EightpuzzleSearchProblem (IMPLEMENTAR) </b>  
Esta es una subclase de SearchProblem donde se definira concretamente el problema de busqueda para el ambiente del 8-puzzle. Se necesita completar Actions (acciones disponibles para un estado dado) y result (que estado resulta de ejecutar una accion en un estado)

In [0]:
class EightpuzzleSearchProblem(SearchProblem):
    
    def __init__(self, initial, goal):
        """El constructor recibe  el estado inicial y el estado objetivo"""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones ejecutables desde el estado state.
        Por ejemplo, para el estado '*12345678' debe retornar: acciones = ['Down', 'Right']"""
        acciones = []
        iblank = state.index('*')    # obtiene el indice del casillero en blanco (representado con *) del estado
        
        ## TODO
        
        return acciones

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion action desde state.
        La accion debe ser alguna de self.actions(state)
        Por ejemplo, para  state='*12345678' y action='Right' debe retornar newState = '1*2345678' """  
        
        iblank = state.index('*')    # obtiene el indice del casillero en blanco (representado con *) del estado
        newState = list(state)          # copia state en una lista newState
        
        ## TODO
        
        

        return ''.join(newState)
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state) 

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion es = 1"""
        return c + 1;

### Probando el entorno 8-puzzle y agente de búsqueda

In [0]:
"""Crea el entorno del 8-puzzle con estado inicial '' """
e = EightpuzzleEnvironment('31264578*')

"""Crea un agente de busqueda para alcanzar goalstate"""
goalstate = '*12345678'
a = Agent( EightpuzzleSearchProgram (goalstate, 'bfs') )

"""Añade el agente creado al entorno"""
e.add_thing(a) 

# Ejecuta el entorno 25 pasos 
e.run(25)

Agente buscara solucion al 8-puzzle: estado_inicial = 31264578*. estado_objetivo=*12345678
Agente planeo una solucion al 8-puzzle con bfs: Seq = ['Left', 'Left', 'Up', 'Up']. Nodos visitados=17. Costo Solucion = 4


# Preguntas:
Responda en base a sus experimentos y relacionando con la teoria:

<b>1) ¿De los tres metodos de busqueda implementados (bfs, dfs, ucs), cuáles garantizan encontrar soluciones optimas? Por Qué?  </b>

<b>2) ¿Cuál o cuales métodos visitan menos nodos para encontrar la solucion? ¿Por Qué? </b>

<b>3) Experimente buscar solucion para el problema con estado inicial = '812\*43765', estado objetivo = '12345678\*'. De una interpretacion de qué esta pasando. </b> 

<b>4) Si la memoria fuera una limitación ¿Qué metodos de busqueda sin informacion, de los experimentados aqui y otros vistos en clase, recomendaria para solucionar cualquier n-puzzle (n=8,15,24)? Justifique </b> 



